In [ ]:
using ITensors
using Plots
using Distributed
using Statistics
addprocs(128);

#include("TrajectorySampler/src/TrajectorySampler.jl")
#using TrajectorySampler
#using TEDdata
#import TEData
#include("TE_data.jl")
#include("functions.jl")
#include("TrajectorySampler.jl")

In [ ]:
Threads.nthreads()

In [5]:
@everywhere begin
    
    include("TEData.jl")
    include("Functions.jl")
    include("TrajectorySampler.jl")


    N = 20
    Jx = 1.
    Jz = 0.
    h = 0.
    gamma = 1.

    dissipation = "Sz"

    maxdim = 700
    cutoff = 1E-8
    conserve_qns = true
    track_states = false
    track_entropy = true
    track_local_observables = ["Sz"]


    # time evolution
    t_end = 100. # end time
    tau = 1. # for data collection
    dt = 0.1 # for discretization

    n_thread = Threads.nthreads()

    v_t = Array((0.: tau: t_end))
    
    var_keys = vcat(["maxdim"], track_local_observables)
    if track_entropy
        var_keys = vcat(["S"],var_keys)
    end

    #data_opt = Array{Any,1}
    #data_bare = Array{Any, 1} 


    # prepare TE
    params = XXZ_data(N, Jx, Jz, h, gamma, dissipation)
    ted = TE_data_XXZ(params, dt; maxdim=maxdim, cutoff=cutoff, conserve_qns=conserve_qns)

    # initial state
    psi = productMPS(ted.s, n-> isodd(n) ? "Up" : "Dn")
end



In [32]:
nsample = Threads.nthreads()

@time @everywhere f_opt() = sample_trajectory(psi, ted, v_t; 
    track_states=track_states, 
    track_entropy=track_entropy, 
    track_local_observables=track_local_observables, 
    optimal=true)

@time @everywhere f_bare() = sample_trajectory(psi, ted, v_t; 
    track_states=track_states, 
    track_entropy=track_entropy, 
    track_local_observables=track_local_observables, 
    optimal=false)

println("Obtaining optimized samples...")
results_opt = pmap(1:nsample) do _
    result=f_opt()
end


println("Obtaining bare samples...")
results_bare = pmap(1:nsample) do _
    result=f_bare()
end

  0.018553 seconds (12.46 k allocations: 506.419 KiB)
  0.019018 seconds (14.18 k allocations: 577.528 KiB)
Obtaining optimized samples...
Obtaining bare samples...


Worker 22 terminated.
Worker 20 terminated.
Worker 7 terminated.
Worker 115 terminated.
Worker 124 terminated.
Worker 5 terminated.
Worker 50 terminated.
Worker 126 terminated.
Worker 4 terminated.
Worker 30 terminated.
Worker 101 terminated.
Worker 48 terminated.
Worker 14 terminated.
Worker 6 terminated.
Worker 116 terminated.
Worker 49 terminated.
Worker 91 terminated.
Worker 118 terminated.
Worker 129 terminated.
Worker 11 terminated.
Worker 105 terminated.
Worker 53 terminated.
Worker 28 terminated.
Worker 103 terminated.
Worker 89 terminated.
Worker 95 terminated.
Worker 33 terminated.
Worker 83 terminated.
Worker 61 terminated.
Worker 128 terminated.
Worker 84 terminated.
Worker 73 terminated.
Worker 8 terminated.
Worker 37 terminated.


LoadError: [91mProcessExitedException(28)[39m

In [39]:
# bring dict labels outwards


function dict_invert(d_in::Array{Dict{Any,Any},1})
    ks = keys(d_in[1])
    N = length(d_in)
    
    # initialize
    d_return = Dict()
    for key in ks
        d_return[key] = Array{Any}[]
    end
    
    for d_sample in d_in
        for key in ks
            push!(d_return[key], d_sample[key] )
        end
    end
    
    return d_return
end
    
d_opt = dict_invert(results_opt)
d_bare = dict_invert(results_bare)    

# get means and save in dictionary
for key in var_keys
    d_opt[key*"_mean"] = mean(d_opt[key])
    d_bare[key*"_mean"] = mean(d_bare[key])
end

In [40]:
label1 = "S_mean"
label2 = "Sz_mean"
time = 19.

it = argmin(abs.(time.-v_t))

l = @layout [a b; c d]
#plot(v_t,d_bare[label1][:,div(N,2)], label="bare")
p1 = plot!(v_t,d_opt[label1][:,div(N,2)], label="optimal")


#plot(v_t,d_bare[label2][:,div(N,2)], label="bare")
p2 = plot!(v_t,d_opt[label2][:,div(N,2)], label="optimal")

#plot(d_bare[label1][end,:], label="bare")
p3 = plot!(d_opt[label1][it,:], label="optimal")

#plot(d_bare[label2][end,:], label="bare")
p4 = plot!(d_opt[label2][it,:], label="optimal")

plot(p1, p2, p3, p4, layout = l, size=(1000,800))


BoundsError: [91mBoundsError: attempt to access 41-element Array{Float64,1} at index [1:101][39m

In [38]:
plot(v_t,mean(d_bare["maxdim"]), label="bare")
plot!(v_t,mean(d_opt["maxdim"]), label="optimal")

BoundsError: [91mBoundsError: attempt to access 41-element Array{Float64,1} at index [1:101][39m

In [ ]:
label = "S_mean"
time = 20.

it = argmin(abs.(time.-v_t))

plot(d_opt[label][it,:], label="optimal")
plot!(d_bare[label][end,:], label="bare")

In [ ]:
label = "Sz_mean"
time = 20.

it = argmin(abs.(time.-v_t))

plot(d_opt[label][it,:], label="optimal")
plot!(d_bare[label][end,:], label="bare")

In [ ]:
offset = 1
println(Array((offset+1):2:(N-1)))
println(Array(reverse((offset+1):2:(N-1))))

In [27]:


function d()
    f(x) = cos(x[1])
    result = optimize(f, [0.])
    return result
end

d()

 * Status: success

 * Candidate solution
    Final objective value:     -1.000000e+00

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    11
    f(x) calls:    25
